In [323]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from  more_itertools import unique_everseen
from datetime import datetime as dt
from urllib.request import *
import re

In [420]:
pd.options.mode.chained_assignment = None

In [249]:
headlinelinks=[]
headlinetitles=[]
timestamps=[]

url = ["https://www.reuters.com/news/archive/mergersNews?view=page&page=1&pageSize=10","https://www.reuters.com/news/archive/EUmergersNews?view=page&page=1&pageSize=10"]

for i in range(20):
    new_page = "page="+str(i)
    for j in url:
        new_url = j
        new_url = new_url.replace("page=1",new_page)
        html = urlopen(new_url)
        soup = bs(html,'html.parser')
        headlines = soup.find_all('div',class_='story-content')
        headlinelinks.extend( [x.find('a')['href'] for x in headlines] )
        headlinetitles.extend( [x.find('h3').text for x in headlines] )
        datetm = [x.find('span',class_='timestamp').text for x in headlines]
        datetm = [re.sub(' EDT','',x) for x in datetm]
        timestamps.extend(datetm)

In [415]:
removelist=['$','million','mln','']
headlinetitles = [x.replace("\t","") for x in headlinetitles]
headlinetitles = [x.replace("\n","") for x in headlinetitles]
#headlinetitles = list(unique_everseen(headlinetitles))
cleanedheadlinetitles = [re.sub("'s","",x) for x in headlinetitles]
cleanedheadlinetitles = [re.sub('[^a-zA-Z|\s]',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(million|mln|billion|bln)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(a|to|be)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s+',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('BRIEF','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('UPDATE','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [x.strip() for x in cleanedheadlinetitles]

In [416]:
cleanedheadlinetitles

['Alphabet CapitalG leads Lyft latest funding',
 'BCB BANCORP REPORTS Q EARNINGS PER SHARE',
 'EU mergers and takeovers Oct',
 'EU mergers and takeovers Oct',
 'Trump names Washington insiders head antitrust consumer protection agency',
 'Unilever sales fall short as competition bites big brands',
 'MOVES Credit Suisse appoints Americas industrials co heads',
 'Chinese funded EV startup buys tech firm headed by former Tesla executive',
 'Tong Oil Tools acquire controlling stake in Cutters Group Management',
 'Deals of the day Mergers and acquisitions',
 'Alphabet CapitalG leads Lyft latest funding',
 'EU mergers and takeovers Oct',
 'EU mergers and takeovers Oct',
 'Trump names Washington insiders head antitrust consumer protection agency',
 'Unilever sales fall short as competition bites big brands',
 'MOVES Credit Suisse appoints Americas industrials co heads',
 'Chinese funded EV startup buys tech firm headed by former Tesla executive',
 'Deals of the day Mergers and acquisitions',


In [ ]:
newtimestamps = []
for item in timestamps:
    try:
        newtimestamps.append(dt.strptime(item,'%b %d %Y'))
    except:
        item = re.sub(r'(am|pm)',r' \1',item)
        currentime = dt.strptime(item,'%I:%M %p')
        currentdate = dt.today()
        currentdate = currentdate.replace(hour = currentime.hour, minute = currentime.minute)
        newtimestamps.append(currentdate)

In [417]:
headlinesframe=[]
headlinesframe = pd.DataFrame(cleanedheadlinetitles,columns=['titles'])
headlinesframe['link'] = pd.DataFrame(headlinelinks)
headlinesframe['datetime'] = pd.DataFrame(newtimestamps)
headlinesframe.sort_values('datetime',ascending=False,inplace=True)
headlinesframededup = headlinesframe.drop_duplicates(subset=['titles'], keep='first')

In [425]:
namelist = pd.read_excel('NameList.xlsx','ModNamesList')
namelistmod = namelist.iloc[:,1:]
searcharray = namelistmod.as_matrix()
newsearcharray=[]
for row in searcharray:
    newrow = [x for x in row if not pd.isnull(x)]
    newsearcharray.append(newrow)

In [394]:
def checknames(headlines):
    rowname = []
    names = []
    for headline in headlines:
        words = headline.split()
        cos = []
        for word in words:
            founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]
            if len(founds)!=0:
                cos.extend(founds)
        names.append([newsearcharray[i][1] for i in cos])
    return names

In [421]:
companeslist = checknames(headlinesframededup['titles'])
headlinesframededup['companies']=companeslist
headlinesframededup                

,titles,link,datetime,companies
30,Alphabet CapitalG leads Lyft latest funding,/article/lyft-ipo-investors/update-2-alphabets...,2017-10-19 13:07:25.743085,"[ALPHABET , ALPHABET ]"
21,BCB BANCORP REPORTS Q EARNINGS PER SHARE,/article/brief-bcb-bancorp-reports-q3-earnings...,2017-10-19 12:11:25.742559,"[BCB , BANCORP ]"
31,EU mergers and takeovers Oct,/article/eu-ma/update-1-eu-mergers-and-takeove...,2017-10-19 12:09:25.743144,[]
33,Trump names Washington insiders head antitrust...,/article/usa-ftc/trump-names-washington-inside...,2017-10-19 11:09:25.743259,[]
34,Unilever sales fall short as competition bites...,/article/unilever-outlook/update-3-unilever-sa...,2017-10-19 10:52:25.743321,[]
35,MOVES Credit Suisse appoints Americas industri...,/article/creditsuisse-industrials/moves-credit...,2017-10-19 10:27:25.743381,[]
36,Chinese funded EV startup buys tech firm heade...,/article/china-autos-sf-motors/chinese-funded-...,2017-10-19 10:00:25.743441,"[TECH , TESLA ]"
28,Tong Oil Tools acquire controlling stake in Cu...,/article/brief-tong-oil-tools-to-acquire-a-con...,2017-10-19 09:10:25.742972,[]
37,Deals of the day Mergers and acquisitions,/article/deals-day/deals-of-the-day-mergers-an...,2017-10-19 09:05:25.743501,[]
40,Genuine Parts announces industrial and automot...,/article/brief-genuine-parts-announces-industr...,2017-10-19 08:44:25.743676,[GENUINE ]


In [426]:
news = ["Agilent to buy Alcoa in a deal","Google in talks to aquire Facebook labs"]
companeslist = checknames(news)
companeslist

[['AGILENT ', 'ALCOA '], ['ALPHABET ', 'FACEBOOK ']]